In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
import sklearn

In [3]:
model = AutoModel.from_pretrained('../BAAI/bge-large-zh-v1.5/')

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda', index=0)

In [6]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [7]:
tokenizer = AutoTokenizer.from_pretrained('../BAAI/bge-large-zh-v1.5/')

In [8]:
import json

In [9]:
questions = json.load(open('./questions.json'))

In [10]:
len(questions)

301

In [11]:
import pdfplumber

In [12]:
pdf = pdfplumber.open('./初赛训练数据集.pdf')

In [13]:
pdf_contents = []
for page_idx, page_content in enumerate(pdf.pages):
    pdf_contents.append({
        'idx': 'page_' + str(page_idx + 1),
        'content': page_content.extract_text()
    })

In [14]:
len(pdf_contents)

354

In [15]:
pdf_contents[0]

{'idx': 'page_1',
 'content': '欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'}

In [16]:
questions_content = [x['question'] for x in questions]

In [17]:
pdf_contents = [x['content'] for x in pdf_contents]

In [18]:
pdf_contents[0]

'欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'

In [19]:
encoded_questions_content = tokenizer(questions_content, return_tensors='pt', padding='max_length', truncation=True, max_length=64)

In [20]:
encoded_questions_content['input_ids'][0]

tensor([ 101,  100, 1184, 2961, 2429, 3488, 6858, 7599,  100, 4638, 4685, 1068,
        1079, 2159, 1762, 5018, 1126, 7552, 8043,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [21]:
max_pdf_content_length = max(len(x) for x in pdf_contents)

In [22]:
max_pdf_content_length

4334

In [23]:
min_pdf_content_length = min(len(x) for x in pdf_contents)

In [24]:
min_pdf_content_length

0

In [25]:
len(pdf_contents)

354

In [26]:
pdf_contents[119]

'安全出行\n01点击按键，设置方向盘加热强度及开关控制。 01点击开启/关闭方向盘自动加热。\n方向盘加热强度分三级调节（低、中、高），点击控制开关后\n可切换方向盘加热强度。 说明！\n使用Lynk&CoApp打开/关闭方向盘加热 □ 车辆启动后，在方向盘自动加热功能开启的情况下，当车内温度\n低于一定值且电量充足时，方向盘加热功能自动启动。\n打开/关闭方向盘加热图标：登录Lynk&CoApp，按下该\n图标可以打开/关闭方向盘加热。\n方向盘加热\n寒冷天气下启用方向盘加热功能，方向盘会逐渐加热到舒适温度，提\n设置方向盘加热时间 高驾驶环境舒适度。\n在中央显示屏中唤起空调控制界面，然后点击 -舒适。\n在中央显示屏唤起空调控制界面。\n120'

In [27]:
encoded_pdf_contents = tokenizer(pdf_contents, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

In [28]:
encoded_pdf_contents['input_ids'].shape

torch.Size([354, 512])

In [29]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [30]:
encoded_pdf_contents = encoded_pdf_contents.to(device)

In [31]:
encoded_questions_content = encoded_questions_content.to(device)

In [32]:
with torch.no_grad():
    questions_content_output = model(**encoded_questions_content)

In [33]:
with torch.no_grad():
    pdf_contents_output = model(**encoded_pdf_contents)

In [35]:
questions_content_output.last_hidden_state.shape

torch.Size([301, 64, 1024])

In [36]:
questions_content_cls= questions_content_output.last_hidden_state[:, 0, :]

In [37]:
pdf_contents_cls = pdf_contents_output.last_hidden_state[:, 0, :]

In [38]:
questions_content_cls.shape

torch.Size([301, 1024])

In [39]:
pdf_contents_cls.shape

torch.Size([354, 1024])

In [43]:
pdf_contents_cls = pdf_contents_cls.detach().cpu().numpy()

In [44]:
questions_content_cls = questions_content_cls.detach().cpu().numpy()

In [48]:
for query_idx, query_content in enumerate(questions_content_cls):
    score = query_content @ pdf_contents_cls.T
    max_score_page = 'page_' + str(score.argsort()[-1] + 1)
    questions[query_idx]['reference'] = max_score_page

In [49]:
with open('submit_2.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

In [50]:
1 + 3.4

4.4